In [1]:
import os
import pandas as pd
import datasets
from datasets import load_dataset

ROOT_DIR = '/mnt/tidal-alsh-hilab/usr/shenguobin/verl/recipe/optimal_sampling/production'
DATA_DIR = f"{ROOT_DIR}/data"

In [12]:
data_local = pd.read_parquet(f'{DATA_DIR}/deepscaler_q14b_q4bb_new/kl_symmetry.parquet')

In [9]:
data = load_dataset("agentica-org/DeepScaleR-Preview-Dataset", split='train')

In [14]:
data[0]

{'problem': 'The operation $\\otimes$ is defined for all nonzero numbers by $a \\otimes b = \\frac{a^{2}}{b}$. Determine $[(1 \\otimes 2) \\otimes 3] - [1 \\otimes (2 \\otimes 3)]$.',
 'answer': '-\\frac{2}{3}',
 'solution': '1. **Apply the operation $\\otimes$ to the innermost parentheses first:**\n   \\[\n   (1 \\otimes 2) \\otimes 3 = \\left(\\frac{1^2}{2}\\right) \\otimes 3 = \\frac{1}{2} \\otimes 3\n   \\]\n   \\[\n   1 \\otimes (2 \\otimes 3) = 1 \\otimes \\left(\\frac{2^2}{3}\\right) = 1 \\otimes \\frac{4}{3}\n   \\]\n\n2. **Calculate each part using the definition of $\\otimes$:**\n   \\[\n   \\frac{1}{2} \\otimes 3 = \\frac{\\left(\\frac{1}{2}\\right)^2}{3} = \\frac{\\frac{1}{4}}{3} = \\frac{1}{12}\n   \\]\n   \\[\n   1 \\otimes \\frac{4}{3} = \\frac{1^2}{\\frac{4}{3}} = \\frac{1}{\\frac{4}{3}} = \\frac{3}{4}\n   \\]\n\n3. **Subtract the two results:**\n   \\[\n   \\left(\\frac{1}{12}\\right) - \\left(\\frac{3}{4}\\right) = \\frac{1}{12} - \\frac{9}{12} = -\\frac{8}{12} = -\\f

In [48]:
print(data_local['prompt'][0])
print(data_local['extra_info'][6]['response'])
print(data_local['extra_info'][6]['original_answer'])

[{'content': 'Question: The operation $\\otimes$ is defined for all nonzero numbers by $a \\otimes b = \\frac{a^{2}}{b}$. Determine $[(1 \\otimes 2) \\otimes 3] - [1 \\otimes (2 \\otimes 3)]$.\n\nAnswer: ', 'role': 'user'}]
<think>
Okay, so I need to figure out how much money you save by buying the appliance from the TV advertiser instead of buying it in-store. The in-store price is $99.99, and the TV deal is three payments of $29.98 each plus a one-time shipping and handling charge of $9.98. I need to find the difference between these two amounts and convert it to cents. Let me break this down step by step.

First, let me calculate the total cost from the TV advertiser. They have three payments of $29.98. So, I need to multiply 29.98 by 3. Let me do that:

29.98 * 3. Hmm, 29.98 is almost 30, so 30 * 3 is 90. But since it's 29.98, which is 0.02 less than 30, I need to subtract 0.02 * 3 from 90. That would be 90 - 0.06 = 89.94. So, the total of the three payments is $89.94.

Then there'

In [37]:
data_local['extra_info'][0].keys()

dict_keys(['alpha_mean', 'alpha_std', 'ess_ratio_mean', 'gpu_rank', 'original_answer', 'original_problem', 'original_solution', 'response', 'sample_idx'])

In [64]:
mixed = load_dataset("/mnt/tidal-alsh-hilab/usr/shenguobin/verl/recipe/optimal_sampling/production/data/deepscaler_q14b_q4bb_new/filtered/kl_symmetry_sft_sampled")['train']
teacher = load_dataset("/mnt/tidal-alsh-hilab/usr/shenguobin/verl/recipe/optimal_sampling/production/data/deepscaler_q14b_q4bb_new/filtered/kl_symmetry_sft_teacher")['train']


In [70]:
n_mix, n_teach = 0, 0

for i in range(len(mixed)):
    n_mix += len(mixed[i]['messages'][-1]['content'])
    n_teach += len(teacher[i]['messages'][-1]['content'])

In [66]:
len(teacher)

21935

In [71]:
n_mix

326345483

In [72]:
n_teach

344742371

### vllm SD test

In [4]:

import os
os.environ["VLLM_USE_V1"] = "0"

from vllm import LLM, SamplingParams

try:
    llm = LLM(
        model="facebook/opt-125m",  # 用小模型测试
        speculative_config={
            "model": "facebook/opt-125m",  # 用自己做 draft
            "num_speculative_tokens": 2,
        },
    )

    outputs = llm.generate("Test", SamplingParams(max_tokens=10))
    print("✅ Draft model IS supported!")
    print(f"Output: {outputs[0].outputs[0].text}")

except Exception as e:
    print(f"❌ Error: {e}")
    if "speculative_config" in str(e):
        print("Hint: Use speculative_config instead of speculative_model")# 测试脚本：验证 draft model 是否支持
from vllm import LLM, SamplingParams

try:
    llm = LLM(
        model="facebook/opt-125m",  # 用小模型测试
        speculative_config={
            "model": "facebook/opt-125m",  # 用自己做 draft
            "num_speculative_tokens": 2,
        },
    )

    outputs = llm.generate("Test", SamplingParams(max_tokens=10))
    print("✅ Draft model IS supported!")
    print(f"Output: {outputs[0].outputs[0].text}")

except Exception as e:
    print(f"❌ Error: {e}")
    if "speculative_config" in str(e):
        print("Hint: Use speculative_config instead of speculative_model")

INFO 11-26 13:19:12 [utils.py:233] non-default args: {'disable_log_stats': True, 'speculative_config': {'model': 'facebook/opt-125m', 'num_speculative_tokens': 2}, 'model': 'facebook/opt-125m'}
INFO 11-26 13:19:13 [model.py:547] Resolved architecture: OPTForCausalLM
INFO 11-26 13:19:13 [model.py:1510] Using max model len 2048
INFO 11-26 13:19:14 [model.py:547] Resolved architecture: OPTForCausalLM
INFO 11-26 13:19:14 [model.py:1510] Using max model len 2048
❌ Error: Speculative decoding with draft model is not supported yet. Please consider using other speculative decoding methods such as ngram, medusa, eagle, or mtp.
INFO 11-26 13:19:14 [utils.py:233] non-default args: {'disable_log_stats': True, 'speculative_config': {'model': 'facebook/opt-125m', 'num_speculative_tokens': 2}, 'model': 'facebook/opt-125m'}
INFO 11-26 13:19:16 [model.py:547] Resolved architecture: OPTForCausalLM
INFO 11-26 13:19:16 [model.py:1510] Using max model len 2048
INFO 11-26 13:19:17 [model.py:547] Resolved ar

In [2]:
!nvidia-smi

Wed Nov 26 13:16:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.8     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L20Z                    On  | 00000000:00:01.0 Off |                    0 |
| N/A   33C    P0             113W / 700W |   1374MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--